In [51]:
import pandas as pd
import numpy as np

# Join results tables together

In [52]:
df_200219=pd.read_csv('200219_results.csv')
df_191213=pd.read_csv('191213g_results.csv')
final_results=pd.concat([df_200219,df_191213], axis=0)
final_results=final_results.reset_index(drop=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
final_results_ps1=final_results[final_results['source']=='PS1']

final_results_ps1[['our_id','RA','Dec']].to_csv('ps1_sources_for_gaia.csv')

# Load in xmatch

In [54]:
xmatch=pd.read_csv('gaia_xmatch_result.csv')

# Cut on astrometric error and proper motion

In [55]:
xmatch['uwe']=np.sqrt(xmatch['astrometric_chi2_al']/(xmatch['astrometric_n_good_obs_al']-5))
print(min(xmatch['uwe']), max(xmatch['uwe']), np.nanmin(xmatch['pmra']), np.nanmax(xmatch['pmdec']))
#and cut on pmra pmdec


0.8511615078820235 166.74739273060956 -75.2187536114139 69.89171461007075


In [56]:
print(np.mean(xmatch['uwe']),np.mean(xmatch['pmra']))

1.9233025757016435 -2.313913060327456


In [57]:
xmatch_cut=xmatch[(xmatch['uwe']>1.5) & (np.abs(xmatch['pmra']<3) | (xmatch['pmra']==np.nan)) & (np.abs(xmatch['pmdec']<3) | (xmatch['pmdec']==np.nan))]

# Recombine gaia-limited df with ps1/GLADE columns

In [60]:
total_cut_df = pd.merge(xmatch_cut, final_results_ps1, how='left', on=['our_id', 'our_id'])

final_results_glade=final_results[final_results['source']=='GLADE']
final_results=pd.concat([total_cut_df,final_results_glade], axis=0)
final_results=final_results.reset_index(drop=True)